<a href="https://colab.research.google.com/github/cds0987/GAN/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class G_block(nn.Module):
    def __init__(self, out_channels, in_channels=3, kernel_size=4, strides=2,
                 padding=1, **kwargs):
        super(G_block, self).__init__(**kwargs)
        self.conv2d_trans = nn.ConvTranspose2d(in_channels, out_channels,
                                kernel_size, strides, padding, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()

    def forward(self, X):
        return self.activation(self.batch_norm(self.conv2d_trans(X)))

In [ ]:
n_G = 64
net_G = nn.Sequential(
    G_block(in_channels=100, out_channels=n_G*8,
            strides=1, padding=0),                  # Output: (64 * 8, 4, 4)
    G_block(in_channels=n_G*8, out_channels=n_G*4), # Output: (64 * 4, 8, 8)
    G_block(in_channels=n_G*4, out_channels=n_G*2), # Output: (64 * 2, 16, 16)
    G_block(in_channels=n_G*2, out_channels=n_G),   # Output: (64, 32, 32)
    nn.ConvTranspose2d(in_channels=n_G, out_channels=3,
                       kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh())  # Output: (3, 64, 64)

In [ ]:
x = torch.zeros((1, 100, 1, 1))
net_G(x).shape

torch.Size([1, 3, 64, 64])

In [ ]:
n_D = 64
net_D = nn.Sequential(
    D_block(n_D),  # Output: (64, 32, 32)
    D_block(in_channels=n_D, out_channels=n_D*2),  # Output: (64 * 2, 16, 16)
    D_block(in_channels=n_D*2, out_channels=n_D*4),  # Output: (64 * 4, 8, 8)
    D_block(in_channels=n_D*4, out_channels=n_D*8),  # Output: (64 * 8, 4, 4)
    nn.Conv2d(in_channels=n_D*8, out_channels=1,
              kernel_size=4, bias=False))  # Output: (1, 1, 1)


In [ ]:

x = torch.zeros((2, 3, 16, 16))
d_blk = D_block(20)
d_blk(x).shape

torch.Size([2, 20, 8, 8])

In [ ]:

x = torch.zeros((1, 3, 64, 64))
net_D(x).shape

torch.Size([1, 1, 1, 1])

In [ ]:
data_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
n_G = 64
net_G = nn.Sequential(
    G_block(in_channels=100, out_channels=n_G*8,
            strides=1, padding=0),  # Output: (64 * 8, 4, 4)
    G_block(in_channels=n_G*8, out_channels=n_G*4),  # Output: (64 * 4, 8, 8)
    G_block(in_channels=n_G*4, out_channels=n_G*2),  # Output: (64 * 2, 16, 16)
    G_block(in_channels=n_G*2, out_channels=n_G),    # Output: (64, 32, 32)
    nn.ConvTranspose2d(in_channels=n_G, out_channels=3,
                       kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh())  # Output: (3, 64, 64)

# Define the Discriminator
n_D = 64
net_D = nn.Sequential(
    D_block(3, n_D),                  # Output: (64, 32, 32)
    D_block(in_channels=n_D, out_channels=n_D*2),   # Output: (64 * 2, 16, 16)
    D_block(in_channels=n_D*2, out_channels=n_D*4), # Output: (64 * 4, 8, 8)
    D_block(in_channels=n_D*4, out_channels=n_D*8), # Output: (64 * 8, 4, 4)
    nn.Conv2d(in_channels=n_D*8, out_channels=1,
              kernel_size=4, bias=False))  # Output: (1, 1, 1)

# Printing out the Generator and Discriminator architectures
print("Generator Architecture:")
print(net_G)
print("\nDiscriminator Architecture:")
print(net_D)

Generator Architecture:
Sequential(
  (0): G_block(
    (conv2d_trans): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (1): G_block(
    (conv2d_trans): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (2): G_block(
    (conv2d_trans): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (3): G_block(
    (conv2d_trans): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

# Define optimizers
optimizer_G = optim.Adam(net_G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(net_D.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training Loop
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net_G.to(device)
net_D.to(device)
net_G.train()
net_D.train()

Sequential(
  (0): D_block(
    (conv2d): Conv2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (1): D_block(
    (conv2d): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (2): D_block(
    (conv2d): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (3): D_block(
    (conv2d): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm): BatchNorm2d(512, eps=1e-05, momentum=0.1,

In [ ]:
import warnings
import torch
import torchvision
from torch import nn

In [ ]:
import torchvision
import torchvision.transforms as transforms

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:05<00:00, 29335581.33it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
import tensorflow as tf

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Filter the dataset to keep only 300 images for the first class
class_to_keep = 0  # Index of the first class
num_images_per_class = 300
indices_to_keep = [i for i, label in enumerate(train_labels) if label == class_to_keep][:num_images_per_class]

# Select only the images and labels that meet the filter criteria
train_images_filtered = train_images[indices_to_keep]
train_labels_filtered = train_labels[indices_to_keep]

# Verify the shape of the filtered dataset
print("Shape of filtered images:", train_images_filtered.shape)
print("Shape of filtered labels:", train_labels_filtered.shape)


170498071/170498071 [==============================] - 6s 0us/step
Shape of filtered images: (300, 32, 32, 3)
Shape of filtered labels: (300, 1)


In [ ]:
print("Shape of filtered images:", train_images_filtered.shape)
print("Shape of filtered labels:", train_labels_filtered.shape)

Shape of filtered images: (300, 32, 32, 3)
Shape of filtered labels: (300, 1)


In [ ]:
class D_block(nn.Module):
    def __init__(self, out_channels, in_channels=3, kernel_size=4, strides=2,
                padding=1, alpha=0.2, **kwargs):
        super(D_block, self).__init__(**kwargs)
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size,
                                strides, padding, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(alpha, inplace=True)

    def forward(self, X):
        return self.activation(self.batch_norm(self.conv2d(X)))

In [ ]:
x = torch.zeros((1, 64, 32, 32))
d_blk = D_block(in_channels=64, out_channels=64*2)
d_blk(x).shape

torch.Size([1, 128, 16, 16])

In [ ]:
n_D = 64
net_D = nn.Sequential(
    D_block(n_D),  # Output: (64, 32, 32)
    D_block(in_channels=n_D, out_channels=n_D*2),  # Output: (64 * 2, 16, 16)
    D_block(in_channels=n_D*2, out_channels=n_D*4),  # Output: (64 * 4, 8, 8)
    D_block(in_channels=n_D*4, out_channels=n_D*8),  # Output: (64 * 8, 4, 4)
    nn.Conv2d(in_channels=n_D*8, out_channels=1,
              kernel_size=4, bias=False),
    nn.Flatten(),  # Flatten the output to apply linear layer
    nn.Linear(1, 1),  # Linear layer to change output shape to (1)
    nn.Sigmoid()  # Apply sigmoid activation
)

In [ ]:
first_image = train_images_filtered[0]  # Assuming you have already filtered the dataset

# Convert the image to a PyTorch tensor and add batch dimension
first_image_tensor = torch.tensor(first_image, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2)



# Print the output
print("Output:", first_image_tensor.shape)

Output: torch.Size([1, 3, 32, 32])


In [ ]:
import torch
import torchvision
from torchvision import datasets
from torch.utils.data import Subset
import torchvision.transforms as transforms

# Define a transformation
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize with mean=0.5, std=0.5
])

# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Filter images from the first class (class index 0: "airplane")
class_index = 0
indices = [i for i, label in enumerate(train_dataset.targets) if label == class_index]

# Keep only 300 images from the first class
subset_indices = indices[:300]

# Create a subset dataset containing only the selected images
subset_dataset = Subset(train_dataset, subset_indices)

# Create a DataLoader for the subset dataset if needed
subset_dataloader = torch.utils.data.DataLoader(subset_dataset, batch_size=64, shuffle=True)



Files already downloaded and verified


In [ ]:
for images, labels in subset_dataloader:
    print(net_D(images).shape)


    break

      # Break after processing the first batch


In [ ]:
x = torch.zeros((64, 3, 64, 64))
net_D(x).shape

torch.Size([64, 1, 1, 1])

In [ ]:
class G_block(nn.Module):
    def __init__(self, out_channels, in_channels=3, kernel_size=4, strides=2,
                 padding=1, **kwargs):
        super(G_block, self).__init__(**kwargs)
        self.conv2d_trans = nn.ConvTranspose2d(in_channels, out_channels,
                                kernel_size, strides, padding, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()

    def forward(self, X):
        return self.activation(self.batch_norm(self.conv2d_trans(X)))

In [ ]:
n_G = 64
net_G = nn.Sequential(
    G_block(in_channels=100, out_channels=n_G*8,
            strides=1, padding=0),                  # Output: (64 * 8, 4, 4)
    G_block(in_channels=n_G*8, out_channels=n_G*4), # Output: (64 * 4, 8, 8)
    G_block(in_channels=n_G*4, out_channels=n_G*2), # Output: (64 * 2, 16, 16)
    G_block(in_channels=n_G*2, out_channels=n_G),   # Output: (64, 32, 32)
    nn.ConvTranspose2d(in_channels=n_G, out_channels=3,
                       kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh())  # Output: (3, 64, 64)


In [ ]:
x = torch.zeros((1, 100, 1, 1))
net_G(x).shape


torch.Size([1, 3, 64, 64])

In [ ]:
net_D(net_G(x)).shape

torch.Size([1, 1, 1, 1])

In [ ]:
import torch.nn as nn

class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, z):
        # Generate fake images from random noise
        fake_images = self.generator(z)

        # Discriminate between real and fake images
        discriminator_output = self.discriminator(fake_images)

        return discriminator_output

# Assuming net_G and net_D are already defined
gan_model = GAN(generator=net_G, discriminator=net_D)


In [ ]:
for images, labels in subset_dataloader:



    break

In [ ]:
import torch.optim as optim

# Define optimizer for both generator and discriminator
generator_optimizer = optim.Adam(net_G.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(net_D.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Number of epochs for training
num_epochs = 1

# Training loop
for epoch in range(num_epochs):
    for real_images, labels in subset_dataloader:
        batch_size = real_images.size(0)

        #######################
        # Train Discriminator #
        #######################

        # Reset gradients
        discriminator_optimizer.zero_grad()

        # Discriminator forward pass with real images
        real_outputs = net_D(real_images)
        real_labels = torch.ones(batch_size, 1)
        real_loss = criterion(real_outputs, real_labels)
        fake_images = net_G(torch.randn(batch_size, 100, 1, 1))
        fake_outputs = net_D(fake_images.detach())  # detach() to prevent gradients flowing back to the generator
        fake_labels = torch.zeros(batch_size, 1)
        fake_loss = criterion(fake_outputs, fake_labels)
        discriminator_loss = real_loss + fake_loss

        # Backpropagation
        discriminator_loss.backward()
        discriminator_optimizer.step()
        generator_optimizer.zero_grad()

        # Generator forward pass with fake images
        fake_outputs = net_D(fake_images)
        generator_loss = criterion(fake_outputs, real_labels)  # Generator tries to fool discriminator

        # Backpropagation
        generator_loss.backward()
        generator_optimizer.step()


        # Generate fake images
        break
